## This file contains all code necessary to time segmentation methods, across multiple scans and different subsets of recordings

In [ ]:
# from pipeline_timing import pipeline_with_dataload
from profiling_util import average_profilings, sort_files, plot_profiling_params
from segmentation.segmentation_pipelines import extract_masks_adapted_time_profiling, save_as_memmap
import suite2p
from pipeline_memray_new import perform_raster_correction, perform_motion_correction
from caiman import load_memmap
import numpy as np
from numpy import std
import os

Run this cell if utils or segmentation_pipelines files get updated

In [ ]:
from importlib import reload
import profiling_util
reload(profiling_util)
from profiling_util import *
import segmentation.segmentation_pipelines
reload(segmentation.segmentation_pipelines)
from segmentation.segmentation_pipelines import extract_masks_adapted_time_profiling, save_as_memmap

### define consolidated functions for running all code necessary for running segmentation pipelines inside of timing wrapper

In [ ]:
def caiman_analysis(scan, num_frames, num_calls, directory_modifier='', params=None):

    if params is None:
        print('Using default caiman params')
        params = {'num_background_components': 1,
        'merge_threshold': 0.7,
        'fps': 8.3091,
        'init_on_patches': True,
        'proportion_patch_overlap': 0.2,
        'num_components_per_patch': 6,
        'init_method': 'greedy_roi',
        'patch_size': [20.0, 20.0],
        'soma_diameter': [3.2, 3.2],
        'num_processes': 8,
        'num_pixels_per_process': 10000}

    print('Performing raster and motion correction for caiman')
    scan = perform_raster_correction(scan, 
                                     temporal_fill_fraction=1, in_place=False)
    scan = perform_motion_correction(scan, in_place=False)
    print('Finished raster and motion correction')

    if not os.path.isdir(f'segmentation/data/caiman/segmentation results{directory_modifier}'):
            os.mkdir(f'segmentation/data/caiman/segmentation results{directory_modifier}')
        
    for frame_length in num_frames:
    
        test_scan = (scan[...,select_middle_frames(scan, frame_length)]).copy()
        save_path = f'segmentation/data/caiman/segmentation results{directory_modifier}/frames{frame_length}'
        if not os.path.isdir(save_path):
            os.mkdir(save_path)
        mmap_filename = save_as_memmap(test_scan, base_name="data/time_profiling_data/caiman_mmaps/caiman").filename
        mmap_scan, (image_height, image_width), num_frames = load_memmap(mmap_filename)
        save_timing_results_file = f'data/time_profiling_data/caiman/{directory_modifier}average_profile_{frame_length}frames_{num_calls}runs.prof'
        profile_function = lambda : extract_masks_adapted_time_profiling(test_scan, mmap_scan, **params)
        result, individual_times, last_output = average_profilings(profile_function, num_calls, save_timing_results_file, save_seg_results=save_path)
        #os.remove(mmap_scan)
        print(f"Averaged {num_calls} function calls, at a time of {round(result.total_tt, 2)} seconds per call. Program run times had a standard deviation of {round(std(individual_times), 3)} seconds.")
        print(f'Saved results in {save_timing_results_file}.')

def s2p_analysis(scan, num_frames, num_calls, directory_modifier='', tau=1.3, fs=8.3091):
    raw_scan = np.moveaxis(scan, -1, 0)
    reg_scan = np.zeros_like(raw_scan)

    # perform suite2p registration
    ops = suite2p.default_ops()
    ops['tau'] = tau
    ops['fs'] = fs
    #ops['data_path'] = ['./segmentation/data/tiffs/']
    #ops['tiff_list'] = ['2scanMovieOneFieldAllFrames.tif']
    ops['save_path0'] = f'segmentation/data/suite2p/segmentation results{directory_modifier}'
    #ops['reg_tif'] = True
    #ops = suite2p.io.tiff_to_binary(ops)
    #f_reg2 = suite2p.io.BinaryFile(ops['Ly'],ops['Lx'], filename=ops['reg_file'], n_frames=ops['nframes'])
    outputs = suite2p.registration_wrapper(f_reg=reg_scan, f_raw=raw_scan, ops=ops)

    ops = suite2p.registration.save_registration_outputs_to_ops(outputs, ops)
    meanImgE = suite2p.registration.compute_enhanced_mean_image(ops['meanImg'].astype(np.float32), ops)
    ops['meanImgE'] = meanImgE

    def run_s2p_save_path(ops, f_reg, classfile, run_num):
        ops['save_path0'] = ops['save_path0'] + f'/test{run_num}'
        if not os.path.isdir(ops['save_path0']):
            os.mkdir(ops['save_path0'])
        output_ops, stat = suite2p.detection_wrapper(f_reg=f_reg, ops=ops, classfile=classfile)
        stat, F, Fneu, F_chan2, Fneu_chan2 = suite2p.extraction_wrapper(stat, f_reg, f_reg_chan2=None, ops=output_ops)
        iscell = suite2p.classify(stat=stat, classfile=classfile)
        dF = F.copy() - output_ops['neucoeff']*Fneu
        dF = suite2p.extraction.preprocess(
            F=dF,
            baseline=output_ops['baseline'],
            win_baseline=output_ops['win_baseline'],
            sig_baseline=output_ops['sig_baseline'],
            fs=output_ops['fs'],
            prctile_baseline=output_ops['prctile_baseline']
        )
        spks = suite2p.extraction.oasis(F=dF, batch_size=output_ops['batch_size'], tau=output_ops['tau'], fs=output_ops['fs'])
    
        # save results
        fpath = output_ops['save_path0']
        np.save(os.path.join(fpath, 'stat.npy'), stat)
        np.save(os.path.join(fpath, 'iscell.npy'), iscell)
        np.save(os.path.join(fpath, 'F.npy'), F)
        np.save(os.path.join(fpath, 'Fneu.npy'), Fneu)
        np.save(os.path.join(fpath, 'spks.npy'), spks)
        np.save(os.path.join(fpath, 'ops.npy'), output_ops)
        print(f'saved ops.npy to {os.path.join(fpath, "ops.npy")}')

    if not os.path.isdir(f'segmentation/data/suite2p/segmentation results{directory_modifier}'):
            os.mkdir(f'segmentation/data/suite2p/segmentation results{directory_modifier}')
    classfile = suite2p.classification.builtin_classfile
    for frame_length in num_frames:

        save_path = f'segmentation/data/suite2p/segmentation results{directory_modifier}/frames{frame_length}'
        if not os.path.isdir(save_path):
            os.mkdir(save_path)
        #ops['frames_include'] = frame_length
        ops['save_path0'] = save_path
        save_timing_results_file = f'data/time_profiling_data/suite2p/{directory_modifier}_average_profile_{frame_length}frames_{num_calls}runs.prof'
        profile_function = lambda index : run_s2p_save_path(ops.copy(), reg_scan[select_middle_frames(reg_scan[:,0,0], frame_length),:,:], classfile, index)
        result, individual_times, last_output = average_profilings(profile_function, num_calls, save_timing_results_file, save_seg_results=False, input_index=True)
        print(f"Averaged {num_calls} function calls, at a time of {round(result.total_tt, 2)} seconds per call. Program run times had a standard deviation of {round(std(individual_times), 3)} seconds.")
        print(f'Saved results in {save_timing_results_file}.')

def caiman_s2p_analysis(filename, frame_lengths, num_calls, directory_modifier='', skip_frames=0, caiman_params=None, s2p_params=(1.3, 8.3091), perform_caiman=True, perform_s2p=True):

    print('Loading in scan')
    if isinstance(filename, str):
        scan = np.load(filename)
    else:
        scan = filename
    if skip_frames > 0:
        print(f'keeping every {skip_frames}th frame')
        scan = scan[:,:,::skip_frames]
    scan -= scan.min()
    

    interpret_last_frame_length = False
    if frame_lengths[-1] < 0:
        interpret_last_frame_length = True
        frame_lengths[-1] = scan.shape[-1]

    if perform_caiman:
        print(f'Starting caiman analysis with frame lengths {frame_lengths}')
        caiman_analysis(scan.copy(), frame_lengths, num_calls, directory_modifier, caiman_params)
        print('Finished caiman analysis')
    if perform_s2p:
        print(f'Starting suite2p analysis with frame lengths {frame_lengths}')
        s2p_analysis(scan, frame_lengths, num_calls, directory_modifier, s2p_params[0], s2p_params[1])
        print('Done')

    if interpret_last_frame_length:
        frame_lengths[-1] = -1
    
def select_middle_frames(scan, return_frames, skip_rows=0, skip_cols=0):
    # Load some frames from the middle of the scan
    num_frames = scan.shape[-1]
    middle_frame = int(np.floor(num_frames / 2))
    frames = slice(max(middle_frame - int(return_frames/2), 0), middle_frame + int(return_frames/2))
    #last_row = -scan.shape[0] if skip_rows == 0 else skip_rows
    #last_col = -scan.shape[1] if skip_cols == 0 else skip_cols
    #mini_scan = scan[skip_rows:-last_row, skip_cols:-last_col, frames]

    return frames

if reading from a pickled dataframe, load it in and pass it to the timing wrapper

In [ ]:
import pickle
# is there any way to read this dataframe in chunks (since it is very large)?
with open('./raster_motion_miniscan_export01.pkl', 'rb') as f:
    data = f.read()
data

## This cell runs the entire wrapper functions defined above

### parameters:
#### filenames: list of files where data you want to analyze are saved. Can also pass in references to numpy arrays if the data is already loaded in
#### frame_lengths: list of positive intgers defining how many frames the segmentation should be performed on. The last entry can be -1 if you want to analyze all frames in the recording
#### num_calls: number of times to perform segmentation (all function calls will be averaged for timing purposes)

In [ ]:
filenames = ['./segmentation/test_scan1.npy', './segmentation/test_scan2.npy', './segmentation/onefield_miniscan.npy']
frame_lengths = [500,1000,2000,3000,5000,7000, 10000,20000,-1]
num_calls = 2
skip_frames = [15,13,11,9,7,5,3,1]
for i in range(2):
    scan = np.load(filenames[i])
    for skip_frame in skip_frames:
        
        caiman_s2p_analysis(scan[:,:,::skip_frame].copy(), frame_lengths=[-1], num_calls=num_calls, skip_frames=0, directory_modifier=f'scan{i+1}skip{skip_frame}')
        #caiman_s2p_analysis(filenames[i], frame_lengths=[-1], num_calls, directory_modifier=f'scan{i+1}')
# do I also need to include fps and calcium indicator data
#for i in range(len(data)):
#    try:
#        caiman_params=data.loc[:, 'params'][i]
#        print('pulled caiman params')
#        print(caiman_params)
#    except Exception:
#        caiman_params=None
#    caiman_s2p_analysis(data.loc[:, 'mini_scan'][i], frame_lengths, num_calls, directory_modifier=f'scan{i+1}', caiman_params=caiman_params) #, caiman_params=data.loc['params'][i]
scan = np.load(filenames[2])
for skip_frame in skip_frames:
    caiman_s2p_analysis(scan[:,:,::skip_frame].copy(), frame_lengths=[-1], num_calls=num_calls, skip_frames=0, directory_modifier=f'skip{skip_frame}')
#caiman_s2p_analysis(filenames[2], frame_lengths, num_calls=5, directory_modifier='')#,'original_file')

In [ ]:
filename = 'segmentation/onefield_miniscan.npy'
frame_lengths = [500,1000,2000,3000,5000,7000, 10000,20000,-1]
num_calls = 5
caiman_s2p_analysis(filename, frame_lengths, num_calls, directory_modifier='', perform_caiman=False)#,'original_file')

Use helper function that will perform segmentation {num_calls} times and average the times. Results will be saved to the directory specified in average_profilings. The cell below contains the basic outline for profiling a function

In [ ]:
num_calls = 5
profile_function = lambda : extract_masks_adapted_time_profiling(test_scan, mmap_scan, **params)
result, individual_times, last_output = average_profilings(profile_function, num_calls, f'data/time_profiling_data/average_profile_{frame_length}frames_{num_calls}runs.prof')
print(f"Averaged {num_calls} function calls, at a time of {round(result.total_tt, 2)} seconds per call. Program run times had a standard deviation of {round(std(individual_times), 3)} seconds.")
print(f'Saved results in time_profiling_data/average_profile_{frame_length}frames_{num_calls}runs.prof.')

The cell below is adapted to profile segmentation of variable length videos performed by CaImAn

The cells below first perform suite2p registration and then profile segmentation by suite2p on variable length videos

In [ ]:
###### verify output of function
import matplotlib.pyplot as plt
masks, traces, background_masks, background_traces, raw_traces = last_output
plt.imshow(masks.sum(axis=-1))

In [ ]:
plot_profiling_params(individual_times, result.total_tt)

In [ ]:
np.save('masks_new.npy', masks)

def print_plain(lines):
    for line in lines:
        print(line.replace('\n', ''))
print_plain([
    "# Timing profiling usage:\n",
    "    - Run \"average_proflings\" function above\n",
    "    - Run \"snakeviz time_profiling_data/average_profile_{num_calls}runs.prof\" to visualize output in browser\n",
    "    - If there are instances of multiple function call timings overlapping with each other, can call \"tuna data/average_profile_{num_calls}runs.prof\" to account for these inconsistencies\n",
    "        - tuna also gives percentages of total run time for each function call\n",
    "    - If using multiprocessing (such as in segmentation), may need to use viztracer: \"viztracer --multi-processing pipeline_timing.py\"\n",
    "        - View results with \"vizviewer --flamegraph result.json\"\n",
    "\n",
    "Memory profiling usage:\n",
    "    - two libraries\n",
    "        - memory_profiler: samples program periodically so you can track memory usage over course of program\n",
    "            - \"mprof run --python python pipeline_mprof.py\"\n",
    "            - \"mprof plot --flame\"\n",
    "                - can get additional granularity into which functions are taking up memory by adding more @profile decorators in pipeline_profiling.py file\n",
    "            - clean up files afterwards with \"mprof clean\"\n",
    "            - can run all steps together with \"python run_mprof.py\"\n",
    "            - can use \"multiprocess\" and \"--include-children\" flags for multiprocess programs\n",
    "        - memray: provides memory size of largest objects at peak memory usage\n",
    "            - \"memray run pipeline_memray.py\"\n",
    "                - use \"--follow-fork\" tag for multiprocessing tracking\n",
    "            - generate graph outlining largest memory allocations during peak memory usage: \"memray flamegraph [output file name from run operator]\"\n",
    "            - open graph in browser: \"open [flamegraph file name]\"\n",
    "            - also outputs interesting memory profiling statistics: \"memray stats [output file name from run operator]\"\n",
    "            - can run all steps together with \"python run_memray.py\"\n",
    "    - can clean up files afterwards by running cell below\n",
    "\n",
    "To run any/all of the above:\n",
    "    - in command line, run \"python run_profiling.py [-m] [-p] [-t] [# of runs to average]\n",
    "        - m runs memray, p runs mprof, and t runs the time profiling\n",
    "        - note: may need to exit out of matplotlib plots to let program continue running (but files also get saved, so no need to worry)\n",
    "\n",
    "Scaling concerns:\n",
    "    - does cProfile work when running with multiple processors?\n",
    "        - we will figure out later\n",
    "        - viztracer definitely works, so may need to switch over to that\n",
    "    - are memory profiling scripts too modularized?\n",
    "        - having too many subfunctions may increase memory consumption\n",
    "        - just keep skeleton of high level functions, similar to how script will be run in real usage\n",
    "        - leave deep modularization to the timing script"
   ])

# Timing profiling usage:
    - Run "average_proflings" function above
    - Run "snakeviz time_profiling_data/average_profile_{num_calls}runs.prof" to visualize output in browser
    - If there are instances of multiple function call timings overlapping with each other, can call "tuna data/average_profile_{num_calls}runs.prof" to account for these inconsistencies
        - tuna also gives percentages of total run time for each function call
    - If using multiprocessing (such as in segmentation), may need to use viztracer: "viztracer --multi-processing pipeline_timing.py"
        - View results with "vizviewer --flamegraph result.json"

Memory profiling usage:
    - two libraries
        - memory_profiler: samples program periodically so you can track memory usage over course of program
            - "mprof run --python python pipeline_mprof.py"
            - "mprof plot --flame"
                - can get additional granularity into which functions are taking up memory by adding more @profile decorators in pipeline_profiling.py file
            - clean up files afterwards with "mprof clean"
            - can run all steps together with "python run_mprof.py"
            - can use "multiprocess" and "--include-children" flags for multiprocess programs
        - memray: provides memory size of largest objects at peak memory usage
            - "memray run pipeline_memray.py"
                - use "--follow-fork" tag for multiprocessing tracking
            - generate graph outlining largest memory allocations during peak memory usage: "memray flamegraph [output file name from run operator]"
            - open graph in browser: "open [flamegraph file name]"
            - also outputs interesting memory profiling statistics: "memray stats [output file name from run operator]"
            - can run all steps together with "python run_memray.py"
    - can clean up files afterwards by running cell below

To run any/all of the above:
    - in command line, run "python run_profiling.py [-m] [-p] [-t] [# of runs to average]
        - m runs memray, p runs mprof, and t runs the time profiling
        - note: may need to exit out of matplotlib plots to let program continue running (but files also get saved, so no need to worry)

Scaling concerns:
    - does cProfile work when running with multiple processors?
        - we will figure out later
        - viztracer definitely works, so may need to switch over to that
    - are memory profiling scripts too modularized?
        - having too many subfunctions may increase memory consumption
        - just keep skeleton of high level functions, similar to how script will be run in real usage
        - leave deep modularization to the timing script
